In [ ]:
import os
import datetime
import pandas as pd
from dateutil.relativedelta import relativedelta
import firebirdsql

# Função para obter a conexão com o banco de dados Firebird


def get_firebird_connection():
    return firebirdsql.connect(
        host=os.getenv('HOST'),
        port=int(os.getenv('PORT', '3050')),
        database=os.getenv('DB_PATH'),
        user=os.getenv('APP_USER'),
        password=os.getenv('PASSWORD'),
        role=os.getenv('ROLE'),
        auth_plugin_name=os.getenv('AUTH'),
        wire_crypt=False,
        charset='ISO8859_1'
    )


# Define o período: de 01/03/2024 a 28/02/2025
start_date = datetime.date(2024, 1, 1)
end_date = datetime.date(2025, 3, 29)

# Conecta ao banco e extrai os dados da tabela APAGAR
conn = get_firebird_connection()
query = """
    SELECT DTPGTO, VALOR
    FROM APAGAR
    WHERE PAGO = 'S' AND
    DTPGTO BETWEEN ? AND ?
    ORDER BY VALOR DESC
"""
df = pd.read_sql(query, conn, params=(start_date, end_date))
conn.close()

# Converte a coluna DTPGTO para datetime (assumindo formato dd/mm/yy)
df['DTPGTO'] = pd.to_datetime(df['DTPGTO'], dayfirst=True)

# Cria uma coluna 'day' com o dia do mês (1 a 31)
df['day'] = df['DTPGTO'].dt.day

# Agrupa os dados por 'day' e calcula a média dos valores para cada dia
avg_by_day = df.groupby('day')['VALOR'].sum().reset_index()

display(avg_by_day)

avg_by_day = avg_by_day.rename(columns={'VALOR': 'avg_valor'})

# Encontra o dia com a menor média de pagamento
min_day = avg_by_day.loc[avg_by_day['avg_valor'].idxmin()]

# Ordena a lista por valor médio
avg_by_day = avg_by_day.sort_values(by='avg_valor', ascending=True)


print("Média de pagamento por dia do mês (geral):")
print(avg_by_day)

print("\nDia com menor pagamento em média:")
print(f"Dia {min_day['day']} com média de {min_day['avg_valor']:.2f}")

/var/folders/gt/g493gb1j08q58kpxyclbc6c40000gn/T/ipykernel_45914/4237174896.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn, params=(start_date, end_date))


,day,VALOR
0,1,217282.11
1,2,305659.66
2,3,376764.56
3,4,430806.31
4,5,382322.60
5,6,200180.26
6,7,234864.27
7,8,126920.09
8,9,119399.39
9,10,161577.24


Média de pagamento por dia do mês (geral):
    day  avg_valor
12   13  115738.99
8     9  119399.39
22   23  124339.03
11   12  126603.31
7     8  126920.09
21   22  135302.07
23   24  139254.62
17   18  141904.42
14   15  149261.17
13   14  151850.72
15   16  157592.85
9    10  161577.24
10   11  162562.96
28   29  164970.44
16   17  173632.06
20   21  177158.57
30   31  179737.25
26   27  195588.88
25   26  197147.98
5     6  200180.26
27   28  201086.26
29   30  215053.44
0     1  217282.11
6     7  234864.27
18   19  241912.90
1     2  305659.66
2     3  376764.56
4     5  382322.60
24   25  395505.03
3     4  430806.31
19   20  485661.20

Dia com menor pagamento em média:
Dia 13.0 com média de 115738.99
